In [35]:
station = pd.read_csv('station.csv')
trip = pd.read_csv('trip.csv')
weather = pd.read_csv('weather.csv')

In [1]:
from __future__ import division
from __future__ import print_function

import numpy as np
import scipy as sp
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

import sklearn as sk
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from patsy import dmatrix, dmatrices

import fancyimpute

import matplotlib as mpl
mpl.rc('font', family='NanumGothic')
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()
pd.options.display.max_columns = 100

import re
import datetime

In [3]:
pd.options.display.max_columns = 100

In [36]:
trip['age'] = 2017 - trip.birthyear
del trip['birthyear']

In [37]:
trip['Date'] = trip.starttime.map(lambda x: re.match(r'\d+/\d+/\d+',x).group())
weather['startdate'] = weather['Date']
station['from_station_id'] = station['station_id']

In [38]:
cycle_share = pd.merge(trip, station, how = 'left', on = 'from_station_id')
cycle_share = pd.merge(cycle_share, weather, how = 'left', on = 'Date')
del cycle_share['startdate']

In [39]:
cycle_share.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,age,Date,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_Pressure_In,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,57.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,47.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain
2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,29.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain
3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,40.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain
4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,46.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain


In [40]:
cycle_share.isnull().sum()

trip_id                            0
starttime                          0
stoptime                           0
bikeid                             0
tripduration                       0
from_station_name                  0
to_station_name                    0
from_station_id                    0
to_station_id                      0
usertype                           0
gender                         89894
age                            89898
Date                               0
station_id                         5
name                               5
lat                                5
long                               5
install_date                       5
install_dockcount                  5
modification_date             171399
current_dockcount                  5
decommission_date             228817
Max_Temperature_F                  0
Mean_Temperature_F               110
Min_TemperatureF                   0
Max_Dew_Point_F                    0
MeanDew_Point_F                    0
M

In [41]:
cycle_share[cycle_share.station_id.isnull()]

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,age,Date,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_Pressure_In,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events
74114,83795,6/2/2015 20:36,6/2/2015 20:43,SEA00093,398.726,Pronto shop,2nd Ave & Blanchard St,Pronto shop,BT-05,Member,Male,24.0,6/2/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,60.0,57,52,50,48,81,68,56,30.07,30.02,29.95,10,10,10,12,8,20,0.0,NaN
193228,210328,5/17/2016 12:03,5/17/2016 12:04,SEA00421,92.313,Pronto shop 2,Pronto shop 2,Pronto shop 2,Pronto shop 2,Member,Male,25.0,5/17/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71,61.0,53,51,49,47,86,64,47,30.30,30.24,30.17,10,10,10,9,4,NaN,0.0,NaN
193333,210449,5/17/2016 15:41,5/17/2016 15:42,SEA00475,65.702,Pronto shop 2,Pronto shop 2,Pronto shop 2,Pronto shop 2,Member,Female,28.0,5/17/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71,61.0,53,51,49,47,86,64,47,30.30,30.24,30.17,10,10,10,9,4,NaN,0.0,NaN
227729,246671,8/12/2016 9:03,8/12/2016 9:06,SEA99997,176.390,8D OPS 02,8D OPS 02,8D OPS 02,8D OPS 02,Short-Term Pass Holder,NaN,NaN,8/12/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87,75.0,63,59,57,55,78,56,36,30.10,30.05,29.98,10,10,10,10,3,NaN,0.0,NaN
227732,246674,8/12/2016 9:06,8/12/2016 9:08,SEA99997,90.845,8D OPS 02,8D OPS 02,8D OPS 02,8D OPS 02,Short-Term Pass Holder,NaN,NaN,8/12/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87,75.0,63,59,57,55,78,56,36,30.10,30.05,29.98,10,10,10,10,3,NaN,0.0,NaN


In [42]:
# 존재하지 않는 station 제거
cycle_share = cycle_share[~cycle_share.station_id.isnull()]

In [11]:
cycle_share.isnull().sum()

trip_id                            0
starttime                          0
stoptime                           0
bikeid                             0
tripduration                       0
from_station_name                  0
to_station_name                    0
from_station_id                    0
to_station_id                      0
usertype                           0
gender                         89892
age                            89896
Date                               0
station_id                         0
name                               0
lat                                0
long                               0
install_date                       0
install_dockcount                  0
modification_date             171394
current_dockcount                  0
decommission_date             228812
Max_Temperature_F                  0
Mean_Temperature_F               110
Min_TemperatureF                   0
Max_Dew_Point_F                    0
MeanDew_Point_F                    0
M

In [72]:
cycle_share[cycle_share.Mean_Temperature_F.isnull()].Date

168373    2/14/2016
168374    2/14/2016
168375    2/14/2016
168376    2/14/2016
168377    2/14/2016
168378    2/14/2016
168379    2/14/2016
168380    2/14/2016
168381    2/14/2016
168382    2/14/2016
168383    2/14/2016
168384    2/14/2016
168385    2/14/2016
168386    2/14/2016
168387    2/14/2016
168388    2/14/2016
168389    2/14/2016
168390    2/14/2016
168391    2/14/2016
168392    2/14/2016
168393    2/14/2016
168394    2/14/2016
168395    2/14/2016
168396    2/14/2016
168397    2/14/2016
168398    2/14/2016
168399    2/14/2016
168400    2/14/2016
168401    2/14/2016
168402    2/14/2016
            ...    
168453    2/14/2016
168454    2/14/2016
168455    2/14/2016
168456    2/14/2016
168457    2/14/2016
168458    2/14/2016
168459    2/14/2016
168460    2/14/2016
168461    2/14/2016
168462    2/14/2016
168463    2/14/2016
168464    2/14/2016
168465    2/14/2016
168466    2/14/2016
168467    2/14/2016
168468    2/14/2016
168469    2/14/2016
168470    2/14/2016
168471    2/14/2016


In [12]:
# 정거장 서비스 종료 날짜
cycle_share.decommission_date.value_counts()

7/2/2016      3461
3/18/2016     2349
8/9/2016       958
10/29/2015     480
Name: decommission_date, dtype: int64

In [13]:
# 정거장 서비스 종료 위치 1
cycle_share[~cycle_share.decommission_date.isnull()].from_station_name.value_counts()

Dexter Ave & Denny Way                       3461
Frye Art Museum / Terry Ave & Columbia St    2349
12th Ave & E Yesler Way                       958
UW McCarty Hall / Whitman Ct                  480
Name: from_station_name, dtype: int64

In [14]:
# 정거장 서비스 종료 위치 2
cycle_share[~cycle_share.decommission_date.isnull()].from_station_id.value_counts()

SLU-18    3461
FH-01     2349
CD-01      958
UW-01      480
Name: from_station_id, dtype: int64

In [15]:
# 정거장 위치 또는 도커 수 변경한 날짜
cycle_share.modification_date.value_counts()

2/24/2015     21776
3/13/2015      8188
2/20/2015      7561
3/4/2015       6402
2/23/2015      4897
11/9/2015      4336
11/2/2015      3461
3/27/2015      2968
3/18/2016      2349
3/24/2015      1290
8/9/2016        958
10/29/2015      480
Name: modification_date, dtype: int64

In [16]:
# 정거장 위치 또는 도커 수 변경한 정거장 1
cycle_share[cycle_share.modification_date.isnull()].from_station_name.value_counts()

Pier 69 / Alaskan Way & Clay St                                           11274
E Pine St & 16th Ave                                                       9466
3rd Ave & Broad St                                                         9392
2nd Ave & Pine St                                                          8198
2nd Ave & Vine St                                                          6568
15th Ave E & E Thomas St                                                   6206
Dexter Ave N & Aloha St                                                    5917
PATH / 9th Ave & Westlake Ave                                              5631
Summit Ave & E Denny Way                                                   5441
12th Ave & E Mercer St                                                     5426
Seattle Aquarium / Alaskan Way S & Elliott Bay Trail                       5385
Summit Ave E & E Republican St                                             5241
2nd Ave & Blanchard St                  

In [17]:
# 정거장 위치 또는 도커 수 변경한 정거장 1
cycle_share[cycle_share.modification_date.isnull()].station_id.value_counts()

WF-01     11274
CH-07      9466
BT-01      9392
CBD-13     8198
BT-03      6568
CH-05      6206
SLU-02     5917
SLU-07     5631
CH-01      5441
CH-15      5426
WF-04      5385
CH-03      5241
BT-05      4986
CH-12      4826
EL-03      4821
SLU-16     4628
PS-04      4585
CBD-05     4540
CH-09      4517
SLU-04     4512
CBD-03     4210
DPD-01     4177
BT-04      3819
FH-04      3365
PS-05      3292
UD-01      3243
CH-06      3204
EL-05      3104
UD-04      3047
CBD-07     2882
CBD-04     2693
ID-04      2182
SLU-20     2134
UD-07      2115
CH-16      1288
UW-10      1160
UD-02      1147
UW-11       988
SLU-21      863
SLU-22      761
WF-03       160
Name: station_id, dtype: int64

In [43]:
cycle_share['from_station_area'] = cycle_share['from_station_id'].map(lambda x: re.match(r'\w*', x).group())

In [44]:
cycle_share['from_station_area'].value_counts()

CH     60990
SLU    53795
CBD    26859
BT     24765
WF     16819
EL     10893
UW     10189
UD      9552
PS      7877
FH      5714
DPD     5467
ID      2182
CD       958
Name: from_station_area, dtype: int64

In [46]:
cycle_share['day'] = cycle_share.Date.map(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y").weekday())

In [47]:
cycle_share['weekend'] = cycle_share['day'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6],
                                                    value = [1, 1, 1, 1, 1, 0, 0])

In [48]:
cycle_share['day'][cycle_share['weekend'] == 0]

1902      5
1903      5
1904      5
1905      5
1906      5
1907      5
1908      5
1909      5
1910      5
1911      5
1912      5
1913      5
1914      5
1915      5
1916      5
1917      5
1918      5
1919      5
1920      5
1921      5
1922      5
1923      5
1924      5
1925      5
1926      5
1927      5
1928      5
1929      5
1930      5
1931      5
         ..
234972    6
234973    6
234974    6
234975    6
234976    6
234977    6
234978    6
234979    6
234980    6
234981    6
234982    6
234983    6
234984    6
234985    6
234986    6
234987    6
234988    6
234989    6
234990    6
234991    6
234992    6
234993    6
234994    6
234995    6
234996    6
234997    6
234998    6
234999    6
235000    6
235001    6
Name: day, dtype: int64

In [49]:
cycle_share.Events.value_counts()

Rain                   74888
Fog                     4402
Fog , Rain              1836
Fog-Rain                1535
Rain , Thunderstorm     1355
Rain-Thunderstorm       1032
Snow                     463
Rain-Snow                252
Rain , Snow               63
Name: Events, dtype: int64

In [50]:
gender_dummy = pd.get_dummies(cycle_share.gender)
gender_dummy.columns = ['Gender_F', 'Gender_M', 'Gender_O']
# 비회원을 결측치로 처리 (0, 0, 0)
gender_dummy[(gender_dummy.Gender_F == gender_dummy.Gender_M) != gender_dummy.Gender_O] = np.nan

In [51]:
gender_dummy

,Gender_F,Gender_M,Gender_O
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
5,0.0,1.0,0.0
6,0.0,1.0,0.0
7,0.0,1.0,0.0
8,1.0,0.0,0.0
9,0.0,1.0,0.0


In [52]:
cycle_share = pd.concat([cycle_share, 
                         dmatrix('from_station_area -1',
                                 data = cycle_share, return_type='dataframe')], 
                        axis = 1)
cycle_share.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,age,Date,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_Pressure_In,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,from_station_area,day,weekend,from_station_area[BT],from_station_area[CBD],from_station_area[CD],from_station_area[CH],from_station_area[DPD],from_station_area[EL],from_station_area[FH],from_station_area[ID],from_station_area[PS],from_station_area[SLU],from_station_area[UD],from_station_area[UW],from_station_area[WF]
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,57.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,47.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,29.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,40.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,46.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
cycle_share = pd.concat([cycle_share, 
                         gender_dummy], 
                        axis = 1)
cycle_share.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,age,Date,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_Pressure_In,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,from_station_area,day,weekend,from_station_area[BT],from_station_area[CBD],from_station_area[CD],from_station_area[CH],from_station_area[DPD],from_station_area[EL],from_station_area[FH],from_station_area[ID],from_station_area[PS],from_station_area[SLU],from_station_area[UD],from_station_area[UW],from_station_area[WF],Gender_F,Gender_M,Gender_O
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,57.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,47.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,29.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,40.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,46.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [54]:
print(cycle_share[cycle_share.Gender_F.isnull()][['trip_id', 'gender', 'age']].isnull().sum())
print()
print(len(cycle_share[cycle_share.Gender_F.isnull()]))

trip_id        0
gender     89892
age        89892
dtype: int64

89892


In [55]:
cycle_share.Events.value_counts()

Rain                   74888
Fog                     4402
Fog , Rain              1836
Fog-Rain                1535
Rain , Thunderstorm     1355
Rain-Thunderstorm       1032
Snow                     463
Rain-Snow                252
Rain , Snow               63
Name: Events, dtype: int64

In [56]:
cycle_share.Events.isnull().sum()

150234

In [57]:
cycle_share.Events = cycle_share.Events.fillna('_')

In [58]:
cycle_share.Events.isnull().sum()

0

In [59]:
cycle_share['Rain'] = cycle_share.Events.replace(to_replace=['Rain', 
                                                      'Fog , Rain', 
                                                      'Fog-Rain', 
                                                      'Rain , Thunderstorm', 
                                                      'Rain-Thunderstorm', 
                                                      'Rain-Snow', 
                                                      'Rain , Snow', 
                                                      'Fog', 
                                                      'Snow', '_'], value=[1, 1, 1, 1 ,1 ,1 ,1, 0, 0, 0])

In [60]:
cycle_share['Snow'] = cycle_share.Events.replace(to_replace=['Rain', 
                                                      'Fog , Rain', 
                                                      'Fog-Rain', 
                                                      'Rain , Thunderstorm', 
                                                      'Rain-Thunderstorm', 
                                                      'Rain-Snow', 
                                                      'Rain , Snow', 
                                                      'Fog', 
                                                      'Snow', '_'], value=[0, 0, 0, 0 ,0, 1 ,1, 0, 1, 0])

In [61]:
cycle_share['Thunderstorm'] = cycle_share.Events.replace(to_replace=['Rain',
                                                              'Fog , Rain',
                                                              'Fog-Rain',
                                                              'Rain , Thunderstorm',
                                                              'Rain-Thunderstorm', 
                                                              'Rain-Snow', 
                                                              'Rain , Snow', 
                                                              'Fog',
                                                              'Snow', '_'], value=[0, 0, 0, 1 ,1 ,0 ,0, 0, 0, 0])

In [62]:
cycle_share['Fog'] = cycle_share.Events.replace(to_replace=['Rain', 
                                                     'Fog , Rain', 
                                                     'Fog-Rain', 
                                                     'Rain , Thunderstorm', 
                                                     'Rain-Thunderstorm', 
                                                     'Rain-Snow', 
                                                     'Rain , Snow', 
                                                     'Fog', 
                                                     'Snow', '_'], value=[0, 1, 1, 0 ,0 ,0 ,0, 0, 1, 0])

In [74]:
# cycle_share = pd.read_csv('cycle_share.csv')
cycle_share.to_csv('cycle_share.csv', index=False)

In [28]:
cycle_share.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,age,Date,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_Pressure_In,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,from_station_area,day,weekend,from_station_area[BT],from_station_area[CBD],from_station_area[CD],from_station_area[CH],from_station_area[DPD],from_station_area[EL],from_station_area[FH],from_station_area[ID],from_station_area[PS],from_station_area[SLU],from_station_area[UD],from_station_area[UW],from_station_area[WF],Gender_F,Gender_M,Gender_O,Snow,Thunderstorm,Fog,Rain
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,57.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,1
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,47.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,1
2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,29.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1
3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,40.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1
4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,46.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,1
